# Pipeline
- 여러 단계(전처리, 변환, 추정)를 연속적으로 연결하여 실행하는 도구
- 전처리와 모델을 하나의 객체로 결합해서 사용 가능
- 교차검증, 하이퍼 파라미터 탐색에 유용
- 데이터의 누수를 방지 : fit_transform( )이 학습 데이터에만 적용되도록 자동 관리

- **매개변수**
    - <span style="color:#ffd33d">**steps**</span>
        </br>: 파이프라인의 단계들을 커스텀한 이름(key)과 객체(value)들을 쌍으로 묶어 표현한 dict 형태
        - 예)<span style="color: #4aff62b0"> [ ( 'scaler', StandardScaler( ) ), ( 'svc', SVC( ) ) ]
        - 기본값: 없음 **(필수 항목이므로)**
    - <span style="color:#ffd33d">**verbose**</span>
        </br>: 각 단계가 실행될 때 로그를 출력할 것인가?
        - 기본값: False

- **속성**
    - <span style="color:#ffd33d">**named_steps**</span>
        </br>: 파이프라인의 각 단계를 딕셔너리형처럼 접근 가능
        - 예)<span style="color: #4aff62b0"> pipe.named_steps['svc']

- **메소드**
    - <span style="color:#ffd33d">**fit(x, y, fit_params)**</span>
        </br>: 각 단계의 fit을 순차적으로 진행
        - scikit-learn의 Pipeline 객체에 포함된 모든 단계를 한 번의 호출로 처리하는 핵심 기능
        - 마지막 단계는 항상 예측 가능한 추정기(모델)여야 한다.
        - 매개변수 fit_params : 파이프라인의 특정 단계에만 추가적인 인수를 전달해야 할 때 유용
    - <span style="color:#ffd33d">**fit_transform(x, y, fit_params)**</span>
        </br>: 각 단계의 fit_transform( )을 순차적으로 진행
        - 전처리(변환기) 단계에서 주로 사용되며, fit(학습)과 transform(변환) 두 가지 작업을 순차적으로 한 번에 수행
        - 마지막 단계는 변환을 시켜주는 클래스
    - <span style="color:#ffd33d">**predict(x)**</span>
        </br>: 마지막 단계에서 predict 함수를 호출

In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
    # GridSearchCV 는 필수
    # 강사님 생각엔 Pipeline 보다 훨씬! 중요하다고 함

In [3]:
# 데이터 로드
# data 폴더 안의 iris.csv 로드
iris = pd.read_csv('../data/iris.csv')
iris.head(1)

,sepal length,sepal width,petal length,petal width,target
0,5.1,3.5,1.4,0.2,Iris-setosa


In [4]:
iris['target'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [5]:
for i, key in enumerate(iris['target'].unique()):
    # print(i)
    # print(key)
    iris['target'] = iris['target'].replace(key, i)

iris['target'].value_counts()

C:\Users\student\AppData\Local\Temp\ipykernel_948\2390836203.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  iris['target'] = iris['target'].replace(key, i)


target
0    50
1    50
2    50
Name: count, dtype: int64

In [6]:
x = iris.drop('target', axis= 1).values
y = iris['target'].values

X_train, X_test, Y_train, Y_test = train_test_split(
    x, y, test_size= 0.3, random_state= 42, stratify= y
)

In [ ]:
# 파이프라인 생성
pipe = Pipeline(
    [
        ('sclaer', StandardScaler()),
        ('svc', SVC())  # probability 매개변수 사용하지 않음
    ],
    verbose= True
)

In [8]:
# pipe를 이용하여 StandardScaler(), SVC() 모델 사용
pipe.fit(X_train, Y_train)

[Pipeline] ............ (step 1 of 2) Processing sclaer, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing svc, total=   0.0s


,steps,"[('sclaer', ...), ('svc', ...)]"
,transform_input,None
,memory,None
,verbose,True
,copy,True
,with_mean,True
,with_std,True
,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'


In [9]:
pred_pipe = pipe.predict(X_test)

In [10]:
# 분류 보고서 출력
print(classification_report(Y_test, pred_pipe))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.88      0.93      0.90        15
           2       0.93      0.87      0.90        15

    accuracy                           0.93        45
   macro avg       0.93      0.93      0.93        45
weighted avg       0.93      0.93      0.93        45



---
# GridSearchCV
- 하이퍼파라미터(매개변수) 조합을 탐색하여 최적의 조합을 찾는 방법
- 각 파라미터 조합별로 교차검증(CV)을 수행해 평균 성능 비교
- 최적의 모델과 성능을 자동으로 제공

- **매개변수**
    - <span style="color:#ffd33d">**estimator**</span>
        </br>: 모델 선택
        - 예)<span style="color:#4aff62b0"> SVC( ) </span>
    - <span style="color:#ffd33d">**param_grid**</span>
        </br>: 탐색할 파라미터의 조합
        - 예)<span style="color:#4aff62b0"> { 'C' : [0, 1, 10], 'kernel' : ['linear', 'rbf'] }</span> 
    - <span style="color:#ffd33d">**cv**</span>
        </br>: 교차 검증 횟수
        - 예) <span style="color:#4aff62b0"> 5</span> -> <span style="color:#4aff62b0">5</span>-폴드 교차 검증 <span style="color:#808080">(5번 반복 검증) </span>
        - 기본값: None
    - <span style="color:#ffd33d">**scoring**</span>
        </br>: 평가 지표 설정
        - 기존에 제공하는 평가 지표와 커스텀하게 생성한 평가 지표도 사용 가능
        - 기본값: None
    - <span style="color:#ffd33d">**refit**</span>
        </br>: 최적의 파라미터로 전체 데이터를 다시 학습할 것인가?
        - 기본값: True
            - True인 경우, best_estimator_ 속성 사용 가능
    - <span style="color:#ffd33d">**error_score**</span>
        </br>: 모델 학습 시 오류가 발생했을 때 어떤 방식으로 처리할 것인가?
        - 기본값: np.nan
            - np.nan : 오류가 발생한 조합은 최종 결과 테이블에서 np.nan으로 표시되며, 오류를 무시하고 계속 탐색
            - "raise" : 모델 학습 중 오류가 발생하는 즉시 GridSearchCV가 멈추고 예외(Exception) 발생
            - 숫자 : 해당 하이퍼파라미터 조합의 점수를 지정된 숫자로 기록하고 계속 탐색
    - <span style="color:#ffd33d">**return_train_score**</span>
        </br>: 교차 검증 시 훈련 성능 점수까지 반영할 것인가?
        - 기본값: False
    - <span style="color:#ffd33d">**verbose**</span>
        </br>: 출력 로그의 수준
        - 기본값: 0
            - 0 : 없음
            - 1 : 간단하게 표시
            - 2 : 상세하게 표시
    - <span style="color:#ffd33d">**n_jobs**</span>
        </br>: CPU 병렬 처리 갯수
        - 기본값: None
            - -1 : 모든 코어를 사용

- **속성**
    - <span style="color:#ffd33d">**cv_results**</span>
        </br>: 각 파라미터 조합별 성능 결과 (훈련/검증 점수, fit 시간 등)
    - <span style="color:#ffd33d">**best_estimator_**</span>
        </br>: 최적의 파라미터로 다시 학습된 모델 객체
    - <span style="color:#ffd33d">**best_params_**</span>
        </br>: 최적의 성능을 낸 파라미터 조합
    - <span style="color:#ffd33d">**best_score_**</span>
        </br>: 최적의 파라미터 조합을 이용한 교차 검증에서의 평균 성능 점수
    - <span style="color:#ffd33d">**refit_time_**</span>
        </br>: 최적의 파라미터로 refit을 하는데 걸린 시간
        - 최적의 성능 뿐만 아니라 시간도 고려해 '적당히 빠르고, 괜찮은 성능'을 찾아야 한다.

- **메소드**
    - <span style="color:#ffd33d">**fit(x, y)**</span>
        </br>: 모든 파라미터 조합에 대해 학습하고 평가해, 최적의 모델을 학습
    - <span style="color:#ffd33d">**predict(x)**</span>
        </br>: 최적의 모델을 이용해 예측
    - <span style="color:#ffd33d">**predict_proba(x)**</span>
        </br>: (분류인 경우) 확률 예측
    - <span style="color:#ffd33d">**score(x, y)**</span>
        </br>: 최적의 모델로 점수를 출력

In [ ]:
# 독립 변수 x, 종속 변수 y 그대로 이용
# 파라미터 탐색에서 사용할 모델을 생성
svc = SVC()

# 최적의 파라미터를 찾기 위한 파라미터 조합
param_grid = {
    'C': [0.1, 1, 10],       # 규제 강도
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# SVC가 분류 모델이므로 평가 지표로 정확도(accuracy) 사용
# 교차 검증의 횟수는 5회
# 아래의 모델 부분을 통해 파이프와 결합할 수도 있음 
grid = GridSearchCV(
    estimator= svc,          # 최적의 파라미터를 만들 모델
    param_grid= param_grid,  # 파라미터의 조합
    cv= 5,                   # 교차 검증 횟수
    scoring= 'accuracy',     # 검증 점수 기준
    verbose= 1,              # 진행 상황을 간단한 로그로 표시
    refit= True              # 전체 데이터 다시 학습
)

# 학습을 통해 최적의 파라미터를 구성
grid.fit(x, y)

# 최적의 파라미터 조합들 확인
print("최적의 파라미터 조합: ", grid.best_params_)
print("최고의 스코어: ", grid.best_score_)
print("최적의 분류 모델: ", grid.best_estimator_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
최적의 파라미터 조합:  {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
최고의 스코어:  0.9800000000000001
최적의 분류 모델:  SVC(C=1, kernel='linear')
